# Binary evolution between two non-degenerate stars

In this notebook we show the features present in the evolution of two non-degenerate, rotating stars

These stars are aimed at being possible progenitors of the high-mass X-ray binary (HMXB) J08408-...

The initial conditions for the binary are that of a circular orbit, with the stars syncronized to the orbital period. The metallicity is assumed to be solar.

### Config stuff

In [ ]:
save_figures = False

In [ ]:
%load_ext autoreload
%autoreload 2

# ---------------------------------------------
# Libraries used
import os
from pathlib import Path
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FixedLocator, MultipleLocator
from matplotlib.collections import LineCollection
from matplotlib import colors
import pandas as pd
from scipy.optimize import bisect

from mesa_reader import MesaInfo
import kipplotter.mkipp as mkipp
import poskiorb

# ---------------------------------------------
# Set some global options
np.seterr(all='ignore')

# ---------------------------------------------
# Load data
root = '..'
run_location = f'{root}/data/raw/star+star'
config_location = f'{root}/config'

# MESA data
star1 = MesaInfo(f'{run_location}/LOGS1/history.data')
star2 = MesaInfo(f'{run_location}/LOGS2/history.data')
binary = MesaInfo(f'{run_location}/LOGS_binary/binary_history.data')

# core-collapse data fnames
star_cc_fname = f'{run_location}/cc_data/star_at_core_collapse.data'
binary_cc_fname = f'{run_location}/cc_data/binary_at_core_collapse.data'

# companion star profile at core collapse
# from log info, the last profile saved for secondary star is profile133
companion_profile_at_cc = MesaInfo(f'{run_location}/LOGS2/profile133.data')

mpl_style = f'{config_location}/style.mpl'
paper_style = f'{config_location}/paper-style.mpl'

# plot config
MTcases  = ['Case A',  'Case B' ]
MTcolors = ['#DEBA93', '#D9B6C4']
mix_zones = ['convection', 'semiconvection', 'thermohaline', 'overshooting']
mix_colors = ['Chartreuse', 'red', 'Gold', 'purple']
mix_hatches = ['///', '\\\\\\\\\\\\', '||', 'xxx']
star1color = '#408191'  # '#D9B6C4'
star2color = '#3F4364'
# starcolors = ['#2D3C52', '#408191']

### Utility functions

In [ ]:
def group_consecutives(vals, step=1):
    '''Return list of consecutive lists of numbers from vals (number list)
    '''
    run = []
    result = [run]
    expect = None
    for v in vals:
        if (v == expect) or (expect is None):
            run.append(int(v))
        else:
            run = [int(v)]
            result.append(run)
        expect = int(v) + step
    return result

def find_tams(center_h1, model):
    for m in model:
        if center_h1[int(m)] < 1e-5:
            return int(m)

---

# Hertzsprung-Russell diagram

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(3.4,3.4), nrows=1, ncols=1)
left, bottom, width, height = [0.65, 0.72, 0.14, 0.14]
ax2 = fig.add_axes([left, bottom, width, height])

# axis labels
ax.set_xlabel('$\\log\,T_{\\rm eff}/{\\rm K}$')
ax.set_ylabel('$\\log\,L/L_\\odot$');

# constant radii lines
x = np.linspace(0, 8, 100)
for r in (1, 10, 100):
    y = 4*x + 2*np.log10(r) - 15.04680947
    ax.plot(x, y, linestyle=':', color='gray', alpha=0.6, zorder=-99)
ax.annotate('$1 \\; R_\\odot$', xy=(4.96, 4.73), color='gray', rotation=290)
ax.annotate('$10 \\; R_\\odot$', xy=(4.465, 4.73), color='gray', rotation=290);

# ZAMS line for Z=0.02
# _, lt, ll = np.loadtxt(fname='zams_hr.data', unpack=True)
# ax.plot(lt, ll, linestyle='--', lw=1, color='black', zorder=-99)

# primary
logTeff = star1.data.log_Teff
logL = star1.data.log_L
pcenterH1 = star1.data.center_h1   # we have pcenter because its needed to match MT phases for secondary HR
pcenterHe4 = star1.data.center_he4
modnum = star1.data.model_number
nm = find_tams(pcenterH1, modnum)

ax.plot(logTeff[100:], logL[100:], color=star1color, label='primary (25 M$_\\odot$)')
ax2.plot(logTeff[100:], logL[100:], color=star1color)

# shade with gray regions of MT phases
# mask = (binary.data.rl_relative_overflow_1 > 0)
mask = binary.data.star_1_radius > binary.data.rl_1
model_numbers = group_consecutives(binary.data.model_number[mask])
for i in range(len(model_numbers)):
    h1 = pcenterH1[int(model_numbers[i][0])]
    he4 = pcenterHe4[int(model_numbers[i][0])]
    if h1 > 1e-6:
        ax.plot(logTeff[model_numbers[i]], logL[model_numbers[i]], color=MTcolors[0], lw=6, zorder=-98)
    elif he4 > 1e-6:
        ax.plot(logTeff[model_numbers[i]], logL[model_numbers[i]], color=MTcolors[1], lw=6, zorder=-98)
    else:
        print('Case C not expected')

# secondary
logTeff = star2.data.log_Teff
logL = star2.data.log_L
modnum = star2.data.model_number

# ax.plot(logTeff[100:-2000], logL[100:-2000], color='orange', label='secondary (22 M$_\\odot$)')
ax.plot(logTeff[100:], logL[100:], color=star2color, label='secondary (22 M$_\\odot$)')
ax2.plot(logTeff[100:], logL[100:], color=star2color)

# mask = (binary.data.rl_relative_overflow_1 > 0)
mask = binary.data.star_1_radius > binary.data.rl_1
model_numbers = group_consecutives(binary.data.model_number[mask])
for i in range(len(model_numbers)):
    h1 = pcenterH1[int(model_numbers[i][0])]
    he4 = pcenterHe4[int(model_numbers[i][0])]
    if h1 > 1e-6:
        ax.plot(logTeff[model_numbers[i]], logL[model_numbers[i]], color=MTcolors[0], lw=6, zorder=-98)
    elif he4 > 1e-6:
        # numbers = model_numbers[i][:-1400]
        ax.plot(logTeff[model_numbers[i]], logL[model_numbers[i]], color=MTcolors[1], lw=6, zorder=-98)
    else:
        print('Case C not expected')

# legends of MT phases
def add_handle(color):
    return patches.Patch(fill=True, facecolor=color, edgecolor=color, linewidth=0, alpha=1)

handles, legends = [], []
for i, case in enumerate(MTcases):
    legends.append(case)
    handles.append(add_handle(MTcolors[i]))
fig.legend(handles, legends,
           handlelength=1.8,
           title='MT phases',
           title_fontsize=9,
           fancybox=True, frameon=True, facecolor='lightgray', edgecolor='dimgray', framealpha=1,
           loc='center', ncol=1, bbox_to_anchor=(0.18,0.15), bbox_transform=ax.transAxes)

ax.legend(loc='upper center', ncol=1, columnspacing=0.6, bbox_to_anchor=(0.27,0.96));

# axis limits & ticks
ax.set_xticks(np.arange(4.3, 5.4,0.2))
ax.xaxis.set_minor_locator(MultipleLocator(0.1))
ax.set_xlim(5.25, 4.35)
ax.set_yticks(np.arange(4.6, 6.2, 0.2))
ax.yaxis.set_minor_locator(MultipleLocator(0.1))
ax.set_ylim(4.7, 5.9)

ax2.set_xticks(np.arange(4.4, 5, 0.05))
ax2.xaxis.set_minor_locator(MultipleLocator(0.025))
ax2.set_xlim(4.575, 4.475)

ax2.set_yticks(np.arange(5.4, 6, 0.05))
ax2.yaxis.set_minor_locator(MultipleLocator(0.025))
ax2.set_ylim(5.42, 5.54)

# reduce fontsize for inset plot
for item in ([ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
             ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(6);
    
# initial orbital period
# ax.annotate('$P_{\\rm orb} = 3$ days', xy=(4.65, 5.72),
#             bbox=dict(facecolor='lightgray', edgecolor='dimgray', boxstyle='round, pad=0.4'));


plt.subplots_adjust(left=0.15);
# if save_figures: plt.savefig('hr.png', dpi=250);

After 5.24 Myr, the initially more massive star overfills its Roche lobe and starts to donate mass to the companion. This mass-transfer (MT) phase is the so-called Case A of MT.

#### brief check of model number in secondary HR diagram

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(2.4,2.4), nrows=1, ncols=1)

# axis labels
ax.set_xlabel('$\\log\,T_{\\rm eff}/{\\rm K}$')
ax.set_ylabel('$\\log\,L/L_\\odot$');

# secondary
logTeff = star2.data.log_Teff
logL = star2.data.log_L
modnum = star2.data.model_number
omega_ratio = star2.data.Omega_div_Omega_crit

ni = 100
nf = 12900 #10850

# Create a set of line segments so that we can color them individually
# This creates the points as a N x 1 x 2 array so that we can stack points
# together easily to get the segments. The segments array for line collection
# needs to be (numlines) x (points per line) x 2 (for x and y)
points = np.array([logTeff, logL]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# Create a continuous norm to map from data points to colors
norm = plt.Normalize(0, 1)  #omega_ratio.min(), omega_ratio.max())
lc = LineCollection(segments, cmap='viridis', norm=norm)

# Set the values used for colormapping
lc.set_array(omega_ratio)
lc.set_linewidth(2)
line = ax.add_collection(lc)
latex = '{\\rm crit}'
fig.colorbar(line, ax=ax, ticks=np.arange(0,1.1,0.2), label=f'$\\Omega/\\Omega_{latex}$')

# axis limits
ax.set_xlim(4.6, 4.4)
ax.set_ylim(5.35, 5.54);

# Timescale for each mass-transfer case

In [ ]:
# apply a mask to use just MT phase values
mask = binary.data.star_1_radius > binary.data.rl_1
numbers = group_consecutives(binary.data.model_number[mask])

time_A = binary.data.age[numbers[0]]
time_B = binary.data.age[numbers[1]]
model_number_A = binary.data.model_number[numbers[0]]
model_number_B = binary.data.model_number[numbers[1]]
star_mass_1_A = star1.data.star_mass[numbers[0]]
star_mass_1_B = star1.data.star_mass[numbers[1]]
star_envelope_mass_1_A = star1.data.star_mass[numbers[0]] - star1.data.he_core_mass[numbers[0]]
star_envelope_mass_1_B = star1.data.star_mass[numbers[1]] - star1.data.he_core_mass[numbers[1]]

print('- Case A:')
print(f'\tt0: {time_A[0]:.5e}, t1: {time_A[-1]:.5e}, dt: {time_A[-1]-time_A[0]:.5e}')
print(f'\tmod_t0: {int(model_number_A[0])}, mod_t1: {int(model_number_A[-1])}')
print(f'\tm1_i: {star_mass_1_A[0]:.2f}, m1_f: {star_mass_1_A[-1]:.2f}, dm1: {star_mass_1_A[0]-star_mass_1_A[-1]:.2f}')
print(f'\tm1_env_i: {star_envelope_mass_1_A[0]:.2f}, m1_env_f: {star_envelope_mass_1_A[-1]:.2f}, dm1: {star_envelope_mass_1_A[0]-star_envelope_mass_1_A[-1]:.2f}')
print('\n- Case B:')
print(f'\tt0: {time_B[0]:.5e}, t1: {time_B[-1]:.5e}, dt: {time_B[-1]-time_B[0]:.5e}')
print(f'\tmod_t0: {int(model_number_B[0])}, mod_t1: {int(model_number_B[-1])}')
print(f'\tm1_i: {star_mass_1_B[0]:.2f}, m1_f: {star_mass_1_B[-1]:.2f}, dm1: {star_mass_1_B[0]-star_mass_1_B[-1]:.2f}')
print(f'\tm1_env_i: {star_envelope_mass_1_B[0]:.2f}, m1_env_f: {star_envelope_mass_1_B[-1]:.2f}, dm1: {star_envelope_mass_1_B[0]-star_envelope_mass_1_B[-1]:.2f}')

# Mass transfer & mass accretion

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, axs = plt.subplots(figsize=(4.2,4.2), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0);

# axis labels
axs[0].set_ylabel('$f_{\\rm mt} = - \\dot{M}_2 / \\dot{M}_1$')
axs[1].set_ylabel('$\\log\,\\mid\\dot{M}\\mid$ [M$_\\odot$ yr$^{-1}$]')
axs[1].set_xlabel('$M_1$ [M$_\\odot$]');

# axis limits & ticks
axs[0].set_yticks(np.arange(0.0, 1.2, 0.2))
axs[0].set_ylim([-0.05, 1.05])
axs[1].set_yticks(np.arange(-8,-1))
axs[1].set_ylim([-8.5, -1.5])
axs[1].set_xticks(np.arange(8, 25, 2))
axs[1].set_xlim([24.5, 8.5]);

# mask data for different MT phases
# mask = binary.data.rl_relative_overflow_1 > 0e0
mask = binary.data.star_1_radius > binary.data.rl_1
numbers = group_consecutives(binary.data.model_number[mask])
mass_1A = binary.data.star_1_mass[numbers[0]]
mdot_1A = np.power(10, binary.data.lg_mstar_dot_1[numbers[0]])
mdot_2A = np.power(10, binary.data.lg_mstar_dot_2[numbers[0]])
# mdotw_1A = np.power(10, binary.data.lg_wind_mdot_1[numbers[0]])
# mdotw_2A = np.power(10, binary.data.lg_wind_mdot_2[numbers[0]])
# mdot_rateA = binary.data.lg_mtransfer_rate[numbers[0]]
mass_1B = binary.data.star_1_mass[numbers[1]]
mdot_1B = np.power(10, binary.data.lg_mstar_dot_1[numbers[1]])
mdot_2B = np.power(10, binary.data.lg_mstar_dot_2[numbers[1]])
# mdotw_1B = np.power(10, binary.data.lg_wind_mdot_1[numbers[1]])
# mdotw_2B = np.power(10, binary.data.lg_wind_mdot_2[numbers[1]])
# mdot_rateB = binary.data.lg_mtransfer_rate[numbers[1]]

# plot
axs[0].plot(mass_1A, mdot_2A/mdot_1A, color='black')
axs[0].plot(mass_1A, mdot_2A/mdot_1A, color=MTcolors[0], lw=6, zorder=-1)
axs[0].plot(mass_1B, mdot_2B/mdot_1B, color='black')
axs[0].plot(mass_1B, mdot_2B/mdot_1B, color=MTcolors[1], lw=6, zorder=-1)

axs[1].plot(mass_1A, np.log10(mdot_1A), color=star1color)
# axs[1].plot(mass_1A, np.log10(mdotw_1A), color=star1color, ls=':')
axs[1].plot(mass_1B, np.log10(mdot_1B), color=star1color, label='$\\dot{M}_1$')
# axs[1].plot(mass_1B, np.log10(mdotw_1B), color=star1color, ls=':')

axs[1].plot(mass_1A, np.log10(mdot_2A), color=star2color)
# axs[1].plot(mass_1A, np.log10(mdotw_2A), color=star2color, ls=':')
axs[1].plot(mass_1B, np.log10(mdot_2B), color=star2color, label='$\\dot{M}_2$')
# axs[1].plot(mass_1B, np.log10(mdotw_2B), color=star2color, ls=':');

# legends of MT phases
def add_handle(color):
    return patches.Patch(fill=True, facecolor=color, edgecolor=color, linewidth=0)

handles, legends = [], []
for i, case in enumerate(MTcases):
    legends.append(case)
    handles.append(add_handle(MTcolors[i]))
axs[0].legend(handles, legends,
              handlelength=1.8,
              title='MT phases', title_fontsize=9,
              fancybox=True, frameon=True,
              facecolor='lightgray', edgecolor='dimgray', framealpha=1.0,
              loc='center', ncol=1,
              bbox_to_anchor=(0.23,0.4), bbox_transform=axs[0].transAxes);
axs[1].legend(loc='center', bbox_to_anchor=(0.3,0.9), ncol=2, bbox_transform=axs[1].transAxes);

At around 18 M$_\odot$ and during the Case A of mass-transfer (MT), the accretor stops to accrete all the material
that reaches its surface, that is why we see a drop in the accretion efficiency ($f_{\rm mt}$)

This same sudden drop in $f_{\rm mt}$ is obtained for the Case B of MT, around 11 M$_\odot$ into the donor star.
In this case, the value of $f_{\rm mt}$ is pretty noisy perhaps due to numerical issues in this faster MT episode

---

# Accretor properties during MT phases

Evolution of rotation rate, luminosity and effective temperature due to mass accretion

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, axs = plt.subplots(figsize=(4.2,3.2), nrows=3, sharex=True)
plt.subplots_adjust(hspace=0)

# inset plot
left, bottom, width, height = [0.47, 0.72, 0.15, 0.15]
ax2 = fig.add_axes([left, bottom, width, height])

# axis labels
axs[0].set_ylabel('$\\Omega / \\Omega_{\\rm crit}$')
axs[1].set_ylabel('$\\log\\,(L/L_\\odot)$')
latex = '{\\rm eff}'
axs[2].set_ylabel(f'$\\log\\,(T_{latex}/K)$')
axs[2].set_xlabel('$M_1$ [M$_\\odot$]');

# data mask
# mask = binary.data.rl_relative_overflow_1 > 0e0
mask = binary.data.star_1_radius > binary.data.rl_1
numbers = group_consecutives(binary.data.model_number[mask])

mass_1A = binary.data.star_1_mass[numbers[0]]
# omega_div_omega_crit_1A = star2.data.surf_avg_omega_div_omega_crit[numbers[0]]
omega_div_omega_crit_1A = star2.data.Omega_div_Omega_crit[numbers[0]]
log_Teff_2A = star2.data.log_Teff[numbers[0]]
log_L_2A = star2.data.log_L[numbers[0]]
log_Lnuc_2A = star2.data.log_Lnuc[numbers[0]]

mass_1B = binary.data.star_1_mass[numbers[1]]
# omega_div_omega_crit_1B = star2.data.surf_avg_omega_div_omega_crit[numbers[1]]
omega_div_omega_crit_1B = star2.data.Omega_div_Omega_crit[numbers[1]]
log_Teff_2B = star2.data.log_Teff[numbers[1]]
log_L_2B = star2.data.log_L[numbers[1]]
log_Lnuc_2B = star2.data.log_Lnuc[numbers[1]];

# plot
axs[0].plot(mass_1A, omega_div_omega_crit_1A, color=star2color)
axs[0].plot(mass_1A, omega_div_omega_crit_1A, color=MTcolors[0], lw=6, zorder=-1)
axs[0].plot(mass_1B, omega_div_omega_crit_1B, color=star2color)
axs[0].plot(mass_1B, omega_div_omega_crit_1B, color=MTcolors[1], lw=6, zorder=-1)

ax2.plot(mass_1B, omega_div_omega_crit_1B, color=star2color, lw=0.5)

axs[1].plot(mass_1A, log_L_2A, color=star2color, label='$\\log\,L$')
axs[1].plot(mass_1A, log_Lnuc_2A, color='black', ls='--', label='$\\log\,L_{\\rm nuc}$')
axs[1].plot(mass_1B, log_L_2B, color=star2color)
axs[1].plot(mass_1B, log_Lnuc_2B, color='black', ls='--')

axs[2].plot(mass_1A, log_Teff_2A, color=star2color)
axs[2].plot(mass_1B, log_Teff_2B, color=star2color);

# axis limits & ticks
axs[0].set_xticks(np.arange(8, 25, 2))
axs[0].xaxis.set_minor_locator(MultipleLocator(1))
axs[0].set_xlim(24.5, 8.5)

axs[0].set_yticks(np.arange(0,1.35,0.25))
axs[0].yaxis.set_minor_locator(MultipleLocator(0.125))
axs[0].set_ylim(0.125, 1.125)

axs[1].set_yticks(np.arange(3,6,0.2))
axs[1].yaxis.set_minor_locator(MultipleLocator(0.1))
axs[1].set_ylim(4.9, 5.7)

axs[2].set_yticks(np.arange(4,5,0.05))
axs[2].yaxis.set_minor_locator(MultipleLocator(0.025))
axs[2].set_ylim(4.425, 4.575);

ax2.set_xticks(np.arange(8.8,9.3,0.05))
ax2.xaxis.set_minor_locator(MultipleLocator(0.025))
ax2.set_xlim(9.22, 9.025)  # 9.12,8.98)
ax2.set_yticks(np.arange(0.8,1.1,0.05))
ax2.yaxis.set_minor_locator(MultipleLocator(0.025))
ax2.set_ylim(0.775, 0.995)  # 0.975)#0.775, 0.97);

# reduce fontsize for inset plot
for item in ([ax2.title, ax2.xaxis.label, ax2.yaxis.label] +
             ax2.get_xticklabels() + ax2.get_yticklabels()):
    item.set_fontsize(4);
    
# legends of MT phases
def add_handle(color):
    return patches.Patch(fill=True, facecolor=color, edgecolor=color, linewidth=0)

handles, legends = [], []
for i, case in enumerate(MTcases):
    legends.append(case)
    handles.append(add_handle(MTcolors[i]))
fig.legend(handles, legends,
          handlelength=1.8,
          title='MT phases', title_fontsize=9,
          fancybox=True, frameon=True,
          facecolor='lightgray', edgecolor='dimgray', framealpha=1.0,
          loc='center', ncol=1,
          bbox_to_anchor=(0.25,0.58)) #, bbox_transform=fig.transAxes);

axs[1].legend(loc='center', bbox_to_anchor=(0.55,0.86), ncol=2);

## Case A of MT: hook to higher L in donor evolution

In [ ]:
log_L_1A = star1.data.log_L[numbers[0]]
amin = np.argmin(log_L_1A)
model_number_of_hook = int(binary.data.model_number[int(numbers[0][amin])])
print(f'model number @ change in slope: {model_number_of_hook}')

### Kippenhahn diagram near hook

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(4,3))

# axis labels
ax.set_xlabel('model number')
ax.set_ylabel('$m/M_\\odot$')

# axis limits
ax.set_xlim([100, model_number_A[-1]])

# kipp plot
args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS1'], history_paths=[f'{run_location}/LOGS1/history.data'],
    xaxis='model_number', time_units='Myr', yaxis='mass', identifier='h1',
    function_on_xaxis = lambda x: x, #np.log10(max_age+0.01 - x),
    contour_colormap=plt.get_cmap('Reds'), log10_on_data=False,
    levels=np.arange(0,1.1,0.1),
    decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

# colobar config
cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax) #, ticks=np.arange(0,7,1))
bar.set_label('$^{1}$H')

# line at luminosity hook
ax.axvline(x=model_number_of_hook, ls=':', color='black')

# legends
# Function to plot lines in label
f = lambda c: plt.plot([],[], color=c, ls=':')[0]
def add_handle(color, hatch):
    return patches.Patch(fill=True, facecolor='none', edgecolor=color, linewidth=1, hatch=hatch, alpha=0.8)

handles, legends = [], []
for i, mix in enumerate(mix_zones):
    legends.append(mix)
    handles.append(add_handle(mix_colors[i], mix_hatches[i]))

fig.legend(handles, legends, loc='center', ncol=2, bbox_to_anchor=(0.52, 0.98));

# Kippenhahn plot for the accretor

## During Case A of MT

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(4,3))

# axis labels
ax.set_xlabel('$t$ [Myr]')
ax.set_ylabel('$m/M_\\odot$')

# axis limits
ax.set_xlim([time_A[0]/1e6, time_A[-1]/1e6])

# kipp plot
args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS2'], history_paths=[f'{run_location}/LOGS2/history.data'],
    xaxis='star_age', time_units='Myr', yaxis='mass', identifier='eps_nuc',
    function_on_xaxis = lambda x: x, #np.log10(max_age+0.01 - x),
    contour_colormap=plt.get_cmap('Reds'), log10_on_data=True,
    levels=np.arange(0,6.1,0.1), decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

# colobar config
cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax, ticks=np.arange(0,7,1))
bar.set_label('$\\log\\,\\epsilon_{\\rm nuc}$')

# legends
# Function to plot lines in label
f = lambda c: plt.plot([],[], color=c, ls=':')[0]
def add_handle(color, hatch):
    return patches.Patch(fill=True, facecolor='none', edgecolor=color, linewidth=1, hatch=hatch, alpha=0.8)

handles, legends = [], []
for i, mix in enumerate(mix_zones):
    legends.append(mix)
    handles.append(add_handle(mix_colors[i], mix_hatches[i]))

fig.legend(handles, legends, loc='center', ncol=2, bbox_to_anchor=(0.52, 0.98));

## During Case B of MT

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(4,3))

# axis labels
ax.set_xlabel('$t$ [$\\times 1000$ yrs]')
ax.set_ylabel('$m/M_\\odot$')

# axis limits
ax.set_xlim(time_B[0]/1e3, time_B[-1]/1e3)
# ax.set_ylim(15,34)

# kipp plot
args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS2'], history_paths=[f'{run_location}/LOGS2/history.data'],
    xaxis='star_age', time_units='yr', yaxis='radius', identifier='he4',
    function_on_xaxis = lambda x: x/1000,
    contour_colormap=plt.get_cmap('Blues'),
    log10_on_data=False,  # True,
    levels=np.arange(0,1.1,0.2),  # np.arange(0,6.1,0.1),
    decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

# colorbar config
cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax) # , ticks=np.arange(0,7,1))
bar.set_label('$^{4}$He')

# legends
# Function to plot lines in label
f = lambda c: plt.plot([],[], color=c, ls=':')[0]
def add_handle(color, hatch):
    return patches.Patch(fill=True, facecolor='none', edgecolor=color, linewidth=1, hatch=hatch, alpha=0.8)

handles, legends = [], []
for i, mix in enumerate(mix_zones):
    legends.append(mix)
    handles.append(add_handle(mix_colors[i], mix_hatches[i]))

fig.legend(handles, legends, loc='center', ncol=2, bbox_to_anchor=(0.52, 0.98));

### Zoom in to region of interest, but showing the rotational rate of the star

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(2.4,2.4))

# axis labels
ax.set_xlabel('model number')
ax.set_ylabel('$r/$R$_\\odot$')

# axis limits
ax.set_xlim(model_number_B[0], model_number_B[-1])
ax.set_ylim(8,19)

# kipp plot
args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS2'], history_paths=[f'{run_location}/LOGS2/history.data'],
    xaxis='model_number', time_units='Myr', yaxis='radius', identifier='omega_div_omega_crit',
    function_on_xaxis = lambda x: x,
    contour_colormap=plt.get_cmap('Blues'),
    log10_on_data=False,
    levels=np.arange(0,1.1,0.1),
    show_rot=True,
    show_semi=False,
    show_conv=False,
    show_therm=False,
    decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

# colorbar config
cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax) # , ticks=np.arange(0,7,1))
latex = '{\\rm crit}'
bar.set_label(f'$\\Omega/\\Omega_{latex}$')

# legends
# Function to plot lines in label
f = lambda c: plt.plot([],[], color=c, ls=':')[0]
def add_handle(color, hatch):
    return patches.Patch(fill=True, facecolor='none', edgecolor=color, linewidth=1, hatch=hatch, alpha=0.8)

handles, legends = [], []
for i, mix in enumerate(mix_zones):
    legends.append(mix)
    handles.append(add_handle(mix_colors[i], mix_hatches[i]))

fig.legend(handles, legends, loc='center', ncol=2, bbox_to_anchor=(0.52, 0.98));

## Kipp plot of accretor up until the collapse of its companion

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(4,3))

# axis labels
ax.set_xlabel('$t$ [Myrs]')
ax.set_ylabel('$m/M_\\odot$')

# axis limits
ax.set_xlim(time_B[-4000]/1e6, star2.data.star_age[-1]/1e6)
ax.set_ylim(0,34)

# kipp plot
args = mkipp.Kipp_Args(
    logs_dirs=[f'{run_location}/LOGS2'], history_paths=[f'{run_location}/LOGS2/history.data'],
    xaxis='star_age', time_units='Myr', yaxis='mass', identifier='eps_nuc',
    function_on_xaxis = lambda x: x,
    contour_colormap=plt.get_cmap('Reds'),
    log10_on_data=True,
    levels=np.arange(0,6.1,0.1),
    show_conv=True,
    show_semi=True,
    show_rot=True,
    show_therm=True,
    show_over=True,
    decorate_plot=False, save_file=False)
kipp_plt = mkipp.kipp_plot(args, fig=fig, axis=ax)

# colorbar plot
cax = plt.axes([0.92, 0.125, 0.03, 0.757])
bar = plt.colorbar(kipp_plt.contour_plot, ax=ax, cax=cax)  # , ticks=np.arange(0,7,1))
latex = '{\\rm nuc}'
bar.set_label(f'$\\log\,\\epsilon_{latex}$');

# legends
# Function to plot lines in label
f = lambda c: plt.plot([],[], color=c, ls=':')[0]
def add_handle(color, hatch):
    return patches.Patch(fill=True, facecolor='none', edgecolor=color, linewidth=1, hatch=hatch, alpha=0.8)

handles, legends = [], []
for i, mix in enumerate(mix_zones):
    legends.append(mix)
    handles.append(add_handle(mix_colors[i], mix_hatches[i]))

fig.legend(handles, legends, loc='center', ncol=2, bbox_to_anchor=(0.52, 0.98));

## Binary conditions at the first SN event

In [ ]:
def match_id(id: str='', fname: str=''):
    '''Search file for id and return value associated to it

    Parameters
    ----------
    id : string
        String to match

    Returns
    -------
    value: float
        Value that matches id
    '''

    with open(fname, 'r') as f:
        for k, line in enumerate(f):
            line = line.strip()
            if line.startswith(id):
                value = float(line.split(' ')[-1])
                return value


def get_collapse_data(star_fname: str, binary_fname: str):
    '''Retrieve information of collapsing binary, from mesabin2dco evolution
    
    Parameters
    ----------
    star_fname : string
        Filename with star data

    binary_fname : string
        Filename with binary data

    Returns
    -------
    CollapseData : dictionary
        Important data needed to compute a grid of post-kick orbital parameters
    '''

    # check for existance of data files
    starpath = Path(star_fname)
    if not starpath.is_file(): raise FileNotFoundError(f'{star_fname} does not exist')

    binarypath = Path(binary_fname)
    if not binarypath.is_file(): raise FileNotFoundError(f'{binary_fname} does not exist')

    StarIdMaps = {
        'm1': 'mass_pre_cc',
        'm1_core_mass': 'c_core_mass_pre_cc',
        'm1_remnant_mass': 'remnant_mass',
        'm1_fallback_fraction': 'fallback_fraction'
    }
    BinaryIdMaps = {
        'm2': 'companion_mass',
        'P': 'period_pre_cc',
        'a': 'separation_pre_cc',
        'r1': 'r_1_pre_cc',
        'r2': 'r_2_pre_cc'
    }

    # get data pre-collapse
    CollapseData = dict()
    for key, value in StarIdMaps.items():
        CollapseData[key] = match_id(value, starpath)

    for key, value in BinaryIdMaps.items():
        CollapseData[key] = match_id(value, binarypath)

    return CollapseData

### load data on core collapse stage

In [ ]:
CollapseData = get_collapse_data(star_cc_fname, binary_cc_fname)

In [ ]:
print(CollapseData)

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, ax = plt.subplots(figsize=(4,4))

# axis labels
ax.set_xlabel('$X$ [R$_\\odot$]')
ax.set_ylabel('$Y$ [R$_\\odot$]')

# axis limits
ax.set_xlim([-90, 90])
ax.set_ylim([-90, 90])

u = 0.0     # x-position of the center
v = 0.0     # y-position of the center
a = CollapseData['a']     # radius on the x-axis
b = CollapseData['a']     # radius on the y-axis

# polar coord
t = np.linspace(0, 2*np.pi, 100)

# plot circular orbit
ax.plot(u+a*np.cos(t), v+b*np.sin(t), color='gray', ls=':', zorder=-99)

# non degenerate star (more massive)
ax.scatter(0, 0, s=CollapseData['r2']*300, c='#FCA90F')

# progenitor of compact object
ax.scatter(-CollapseData['a'], 0, s=CollapseData['r1']*300, c='#2B8DD2', zorder=-98)

# compact object
ax.scatter(-CollapseData['a'], 0, s=50, c='black');

# text on conditions at CC
latex = '{\\rm orb}'
collapse_text = f"         Conditions at CC\n"
collapse_text += f"$a$ = {CollapseData['a']:.2f} R$_\\odot$, "
collapse_text += f"$P_{latex} = {CollapseData['P']:.2f}$ days\n"
collapse_text += f"$m_1$ = {CollapseData['m1']:.2f} M$_\\odot$"
collapse_text += f"$\\rightarrow$ {CollapseData['m1_remnant_mass']:.2f} M$_\\odot$\n"
collapse_text += f"$m_2$ = {CollapseData['m2']:.2f} M$_\\odot$"
ax.annotate(collapse_text,
            fontsize=8,
            bbox=dict(facecolor='lightgray', edgecolor='dimgray', boxstyle='round, pad=0.4'),
            xy=(0.35,0.7), xycoords='figure fraction'
);

### Companion star at core collapse

In [ ]:
companion_profile_at_cc.data_dict.keys()

In [ ]:
# plot config
if save_figures:
    plt.style.use(paper_style)
else:
    plt.style.use(mpl_style)
fig, axs = plt.subplots(figsize=(4.2,3.2), nrows=2, sharex=True)
plt.subplots_adjust(hspace=0)

# axis labels
latex = '{\\rm crit}'
axs[1].set_xlabel('mass [M$_\\odot$]')
axs[0].set_ylabel(f'$\\Omega/\\Omega_{latex}$')
axs[1].set_ylabel('abundances')

# global xlims
axs[1].set_xlim(0,33)
axs[1].set_yscale('log')

# rotation profile
axs[0].plot(companion_profile_at_cc.data.mass, companion_profile_at_cc.data.omega_div_omega_crit);
axs[0].axhline(y=star2.data.surf_avg_omega_div_omega_crit[0], color='black', ls=':')

# composition profile
axs[1].plot(companion_profile_at_cc.data.mass, companion_profile_at_cc.data.h1, color='C0')
axs[1].plot(companion_profile_at_cc.data.mass, companion_profile_at_cc.data.he4, color='C1')
axs[1].plot(companion_profile_at_cc.data.mass, companion_profile_at_cc.data.c12, color='C2')
axs[1].plot(companion_profile_at_cc.data.mass, companion_profile_at_cc.data.n14, color='C3')

# ticks params & labels
yticks = np.arange(0,1,0.1)
axs[0].set_yticks(yticks)
axs[0].yaxis.set_minor_locator(MultipleLocator(0.05))
axs[0].set_ylim(0, 0.55)

yticks = [1e-4, 1e-3, 1e-2, 1e-1, 1]
axs[1].set_yticks(yticks)
axs[1].set_ylim([5e-5, 5])
axs[1].yaxis.set_ticklabels(['$10^{-4}$', '$10^{-3}$', '$10^{-2}$', '$0.1$', '$1$'])

axs[1].annotate("$^{1}$H", xy=(0.5,0.15), color='C0', fontsize=7)
axs[1].annotate("$^{4}$He", xy=(0.5,1), color='C1', fontsize=7)
axs[1].annotate("$^{12}$C", xy=(0.5,2.7e-4), color='C2', fontsize=7)
axs[1].annotate("$^{14}$N", xy=(0.5,0.014), color='C3', fontsize=7);